In [4]:
from main import *
from pydantic import ValidationError
from pprint import pprint
repo = github_client.get_repo("ktrnka/company-detective")

try:
    result = review_pull_request(repo, repo.get_pull(17), use_base_readme=True)
    print(result)
except ValidationError as e:
    pprint(e.json)


should_update=False reason="The current README is comprehensive and up-to-date. The pull request changes are primarily focused on internal code modifications and don't introduce any new features or significant changes that would require updates to the README. The existing README already covers the project's purpose, features, prerequisites, installation instructions, and usage information adequately." updated_readme=None


In [5]:
print(f"""
Should update: {result.should_update}

Reason:
{result.reason}

Updated README:
{result.updated_readme}
""")


Should update: False

Reason:
The current README is comprehensive and up-to-date. The pull request changes are primarily focused on internal code modifications and don't introduce any new features or significant changes that would require updates to the README. The existing README already covers the project's purpose, features, prerequisites, installation instructions, and usage information adequately.

Updated README:
None



In [7]:
# Show the readme diff
import difflib

if result.should_update:
    diff = difflib.unified_diff(
        repo.get_readme().decoded_content.decode("utf-8").splitlines(),
        result.updated_readme.splitlines(),
    )
    for line in diff:
        print(line)
